In [5]:
import requests
import datetime

import json
import pandas as pd
pd.set_option('display.max_rows', 2000)

## Get all district IDs

In [6]:
district_df_all = None
for state_code in range(1,40):
    response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_code))
    district_df = pd.DataFrame(json.loads(response.text))
    district_df = pd.json_normalize(district_df['districts'])
    if district_df_all is None:
        district_df_all = district_df
    else:
        district_df_all = pd.concat([district_df_all, district_df])
        
    district_df_all.district_id = district_df_all.district_id.astype(int)
        
district_df_all = district_df_all[["district_name", "district_id"]].sort_values("district_name")
district_df_all.to_csv("districts.csv", index=False)

## Fix your district id 

In [7]:
Ahmedabad = 154
Ahmedabad_Corporation = 770
district_ids = [Ahmedabad, Ahmedabad_Corporation]
next_n_days = 3

In [10]:
base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(next_n_days)]
date_str = [x.strftime("%d-%m-%Y") for x in date_list]
INP_DATE = date_str[-1]

all_date_df = None

for DIST_ID in district_ids:
    print(f"checking for INP_DATE:{INP_DATE} & DIST_ID:{DIST_ID}")
    URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, INP_DATE)
    response = requests.get(URL)
    data = json.loads(response.text)['centers']
    df = pd.DataFrame(data)
    df = df.explode("sessions")
    df['min_age_limit'] = df.sessions.apply(lambda x: x['min_age_limit'])
    df['available_capacity'] = df.sessions.apply(lambda x: x['available_capacity'])
    df['date'] = df.sessions.apply(lambda x: x['date'])
    df = df[["date", "min_age_limit", "available_capacity", "pincode", "name", "state_name", "district_name", "block_name", "fee_type"]]
    if all_date_df is not None:
        all_date_df = pd.concat([all_date_df, df])
    else:
        all_date_df = df
            
df = df.drop(["block_name"], axis=1).sort_values(["min_age_limit", "available_capacity"], ascending=[True, False])
df[df.min_age_limit < 45]

checking for INP_DATE:03-05-2021 & DIST_ID:154
checking for INP_DATE:03-05-2021 & DIST_ID:770


,date,min_age_limit,available_capacity,name,state_name,district_name,pincode,fee_type
33,04-05-2021,18,100,Kendriya Vidhyalaya,Gujarat,Ahmedabad Corporation,380004,Free
33,05-05-2021,18,100,Kendriya Vidhyalaya,Gujarat,Ahmedabad Corporation,380004,Free
33,06-05-2021,18,100,Kendriya Vidhyalaya,Gujarat,Ahmedabad Corporation,380004,Free
33,07-05-2021,18,100,Kendriya Vidhyalaya,Gujarat,Ahmedabad Corporation,380004,Free
33,08-05-2021,18,100,Kendriya Vidhyalaya,Gujarat,Ahmedabad Corporation,380004,Free
36,05-05-2021,18,100,Khodiyar Nagari School No.-22,Gujarat,Ahmedabad Corporation,380022,Free
36,06-05-2021,18,100,Khodiyar Nagari School No.-22,Gujarat,Ahmedabad Corporation,380022,Free
36,07-05-2021,18,100,Khodiyar Nagari School No.-22,Gujarat,Ahmedabad Corporation,380022,Free
6,03-05-2021,18,99,Municipal School No13 I Colony,Gujarat,Ahmedabad Corporation,380024,Free
6,04-05-2021,18,99,Municipal School No13 I Colony,Gujarat,Ahmedabad Corporation,380024,Free
